In [1]:
import os
import openai

In [2]:
# os.environ['OPENAI_API_KEY'] = ''

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [27]:
def create_test_prompt(topic: str, num_questions: int, num_possible_answers: int):
    try:
        num_questions = int(num_questions)
        num_possible_answers = int(num_possible_answers)
        if num_questions < 1 or num_possible_answers < 2:
            raise Exception("The number of questions must be greater or equal to 1 and the number of possble answers must be greater or equal to 2!")
        prompt = f"Create a multiple choice quiz on the topic of {topic}, consisting of {num_questions} questions. "\
            +f"Each question should have {num_possible_answers} options labelled by letters. "\
            +f"Please make sure the questions and answers you provide are backed up by credible sources but you do not need to provide them. "\
            +f"Also include the correct answer for each question using the starting string 'Correct Answer:'"
            
        return prompt
    except ValueError:
        print("Your argument contained non-integer values for num_questions or num_possible answers")

In [41]:
response = openai.Completion.create(engine='text-davinci-003',prompt=create_test_prompt('Java OOP',4,4),max_tokens=256,temperature=0.7)

In [45]:
def create_student_view(test, num_questions):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line + '\n'
        else:
            if question_number < num_questions:
                question_number += 1
                student_view[question_number] = ''
    return student_view

In [48]:
result = create_student_view(response['choices'][0]['text'], 4)

In [58]:
for key in result:
    print(result[key])



Q1. What is the keyword used for declaring a class in Java?

A. method
B. variable
C. function
D. class



Q2. What is the default value of a static variable in Java?

A. null
B. 0
C. false
D. true



Q3. What is the keyword used to create a constructor in Java?

A. constructor
B. new
C. set
D. initialize



Q4. What is the purpose of the instanceof keyword in Java OOP?

A. To access a particular instance of an object
B. To determine the type of an object
C. To create a new instance of an object
D. To assign a certain value to an object




In [57]:
def extract_answers(test, num_questions):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += "Q" + str(question_number) + " "
            answers[question_number] += line + '\n'
            if question_number < num_questions:
                question_number += 1
                answers[question_number] = ''
    return answers

In [59]:
ans = extract_answers(response['choices'][0]['text'], 4)

In [60]:
for key in ans:
    print(ans[key])

Q1 Correct Answer: D. class

Q2 Correct Answer: B. 0

Q3 Correct Answer: A. constructor

Q4 Correct Answer: B. To determine the type of an object



In [54]:
def take_exam(student_view):
    student_answers = {}
    for num, question in student_view.items():
        print(question)
        answer = input("Enter your answer in letters: ")
        student_answers[num] = answer.upper()
    return student_answers

In [ ]:
student_answers = take_exam(create_student_view(response['choices'][0]['text'], 4))

In [56]:
print(student_answers)

{1: 'D', 2: 'A', 3: 'B', 4: 'B'}


In [69]:
def mark_exam(student_answers: dict, answer_key: dict):
    total_marks = len(answer_key)
    student_mark = 0
    true_or_false = []
    for question,answer in student_answers.items():
        if answer == answer_key[question][answer_key[question].index('Correct Answer:')+16]:
            student_mark += 1
            true_or_false.append("Correct")
        else:
            true_or_false.append("Incorrect")
    grade = 100*(student_mark/total_marks)
    if grade < 50:
        passed = "FAIL"
    elif 50 <= grade <= 60:
        passed = "D"
    elif 60 <= grade <= 70:
        passed = "C"
    elif 70 <= grade <= 80:
        passed = "B"
    else:
        passed = "A"
    return f"You have answered {student_mark} out of {total_marks} questions correctly. "\
            +f"Your grade for this test is {passed}, {grade}%. Here are which questions you answered correctly or incorrectly: "\
            + str(true_or_false)

In [70]:
mark_exam(student_answers, ans)

"You have answered 2 out of 4 questions correctly.Your grade for this test is D, 50.0%. Here are which questions you answered correctly or incorrectly: ['Correct', 'Incorrect', 'Incorrect', 'Correct']"